# Проект 'Классификация комментариев'

## Подготовка

### Импорт библиотек

In [1]:
import numpy as np
import pandas as pd
import re 
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from tqdm.auto import tqdm

from pandarallel import pandarallel

### Загрузка датасета

In [4]:
try:
    df = pd.read_csv("datasets/toxic_comments.csv", index_col=0)
except:
    df = pd.read_csv("toxic_comments.csv", index_col=0)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [6]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


В первом стобце мы видим непонятные значения (скорее всего это индексы, в дальнейшем они нам не понадобятся). Также можно заметить, что комментарии на **английском языке**. 

### Создание признаков

В исходном виде модели не воспримут имеющуюся информацию. Для его анализа на эмоционально окрашнные слова сначала **лемматизируем текст**, затем по корпусам рассчитаем **TF-IDF**.

#### Лемматизация

Создадим функцию для очищения текста:

In [7]:
def clear_text(text):
    return " ".join(re.sub(r'[^a-zA-Z ]', ' ', text).lower().split())

Применим её и поместим результат в столбец **'clear_text'**.

In [8]:
df['clear_text'] = df['text'].apply(clear_text)

Для лемматизации воспользуемся пакетом **spaCy**. \
Напишем функцию для лемматизации текста: 

In [18]:
def lemmatize(text):
    # имортируем библиотеку spacy именно здесь,
    # так как далее функция будет выполняться 
    # с помощью параллельных вычислений, а библиотека
    # pandarallel имеет особенности работы на Windows
    import spacy
    sp = spacy.load('en_core_web_sm')
    return " ".join([token.lemma_ for token in sp(text)])


Проверим лемматизатор для 1-й строчки датасета:

In [19]:
%%time 

# lemmatize a sentence
sentence = df.loc[0, 'clear_text']
print(lemmatize(sentence))

explanation why the edit make under my username hardcore metallica fan be revert they weren t vandalism just closure on some gas after I vote at new york dolls fac and please don t remove the template from the talk page since I m retire now
CPU times: total: 781 ms
Wall time: 774 ms


Работает корректно, применим к датасету. Поместим результат в столбец **'lemm_text'**:

In [20]:
%%time

# tqdm.pandas()
tqdm.pandas(desc="progress")
pandarallel.initialize(progress_bar = True)
df['lemm_text'] = df['clear_text'].parallel_apply(lemmatize)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


CPU times: total: 2min 32s
Wall time: 4h 6min 21s


Взглянем на данные:

In [21]:
df.head(3)

,text,toxic,clear_text,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...,explanation why the edit make under my usernam...
1,D'aww! He matches this background colour I'm s...,0,d aww he matches this background colour i m se...,d aww he match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...,hey man I m really not try to edit war it s ju...


Можно заметить, что преобразовались далеко не все слова. Эту проблему можно исправить, добавив функцию определения части речи, но это существенно замедлит процесс. Поэтому будем работать с таким корпусом.

Теперь нас будут интересовать два столбца: **lemm_text** (по нему сформируем TF-IDF) и **toxic** (целевой признак).

#### Разделение на выборки

Из датасета выделим обучающую, валидационную и тестовую выборки:

In [22]:
train, test = train_test_split(df, stratify=df['toxic'], test_size=0.25, random_state=12345)

features_train = train['lemm_text']
target_train = train['toxic']

features_test = test['lemm_text']
target_test = test['toxic']

#### TF-IDF

Укажем стоп-слова (они не будут учитываться при расчёте TF-IDF).

In [23]:
stopwords = stopwords.words('english')

Создадим счётчик:

In [51]:
tfidf_vect= TfidfVectorizer(stop_words=stopwords)	

Посчитаем TF-IDF для корпусов:

In [53]:
tf_idf_train = tfidf_vect.fit_transform(features_train)
tf_idf_test = tfidf_vect.transform(features_test)

In [54]:
print(tf_idf_train.shape)
print(tf_idf_train[0])

(119469, 127877)
  (0, 82192)	0.14503539277098515
  (0, 109900)	0.15009693926571613
  (0, 93629)	0.20176327782300096
  (0, 67330)	0.16461216614546184
  (0, 89958)	0.31496432165615434
  (0, 17929)	0.29467213127582514
  (0, 119396)	0.5039259503987833
  (0, 834)	0.6698167703137718


### Вывод

Мы загрузили датасет и все необходимые библиотеки. Текстовые данные были подготовлены и помещены в 2 выборки для обучения и проверки моделей. \
Теперь выясним, какая из них лучше справится с задачей классификации комментариев.

## Обучение

### DecisionTreeClassifier

Начнём с дерева решений.

In [28]:
%%time

tree_model = DecisionTreeClassifier(random_state=12345)

# подберем оптимальные параметры глубины дерева
parameters_1 = {'max_depth': [5, 10, 15]}

grid_tree = GridSearchCV(tree_model,
                         parameters_1, 
                         cv=3, 
                         scoring='f1')

grid_tree.fit(tf_idf_train, target_train)

grid_tree_params = grid_tree.best_params_
grid_tree_score = grid_tree.best_score_

# лучшее значение F1 на кросс-валидации
print(f'best_score: {round(grid_tree_score, 2)}')
# лучшие гиперпараметры
print(f'best_params: {grid_tree_params}')

best_score: 0.64
best_params: {'max_depth': 15}
CPU times: total: 2min 32s
Wall time: 2min 32s


Значение метрики пока маленькое, будем продолжать.

### LogisticRegression

Обучим логистическую регрессию.

In [29]:
%%time

regr_model = LogisticRegression(fit_intercept=True, 
                                class_weight='balanced', 
                                random_state=12345,
                                solver='liblinear'
                               )

# параметры
parameters_2 = [{'max_iter': [50, 100]}, 
                {'C': [0.1, 1, 10]}]

grid_regr = GridSearchCV(regr_model,
                         parameters_2, 
                         cv=3, 
                         scoring='f1')

grid_regr.fit(tf_idf_train, target_train)

grid_regr_params = grid_regr.best_params_
grid_regr_score = grid_regr.best_score_

# лучшее значение F1 на кросс-валидации
print(f'best_score: {round(grid_regr_score, 2)}')
# лучшие гиперпараметры
print(f'best_params: {grid_regr_params}')

best_score: 0.76
best_params: {'C': 10}
CPU times: total: 15 s
Wall time: 14.9 s


Метод **fit** работает **3 минуты**:

Метрика достигла необходимого значения!

### Проверка на тестовых данных

Наилучшую модель (**LogisticRegression**) проверим на тестовой выборке:

In [31]:
test_pred_regr = grid_regr.predict(tf_idf_test)
f1_regr_test = f1_score(target_test, test_pred_regr)
print(f'F1 LogisticRegression на тестовой выборке: {round(f1_regr_test, 2)}')

F1 LogisticRegression на тестовой выборке: 0.77
